# Pryngles module: System

In [1]:
from pryngles import *

## External modules

In [2]:
import rebound as rb

## System Class

This is the most important class in the whole package.  This class allows to create the planetary system and manipulate it.

In [6]:
System_doc=\
f"""
Creates a planetary system.

Initialization attributes:

    units: list of strings, default = ['au','msun','yr']:
        Units used in calculations following the conventions and signs of rebound.
        The order SHOULD always be MKS: length, mass, time (in that order)
        
    stars: list or single Body.  default = None: 
        Star(s) in the system.
        
    planets: list or single Body.  default = None: 
        Planet(s) in the system.
        
    rings: list or single Body.  default = None: 
        Ring(s) in the system.

    observers: list or single Body.  default = None: 
        Observer(s) in the system.
    
    rebound: boolean, default = True:
        Set True if you want to simulte the orbit of objects (for instance if you want to calculate
        TTVs or TDVs) or False if you want to calculate orbits using pure Keplerian dynamics (no TTVs or TDVs).
        
        NOTE: The current version ({version}) does not implement yet rebound simulations

Secondary attributes:

    hashes: list
        List of hashes of bodies in the system.
        
    stars, planets, rings, observers: list
        List of the corresponding kind of object in the system.
        
    nstars, nplanets, nrings, nobservers: int
        Number of each kind of body in the system.
    
    nbodies: int
        Number of bodies in the system.
        
    ul, um, ut: float [SI units]
        Value of the conversion factors for each unit.
        
    G: float [ul^3/ut^2/um]
        Value of the gravitational constant.

    sim: rebound Simulation.
        Rebound simulation object.

Examples:

    S=Star()
    P=Planet(primary=S)

    sys=System()
    sys=System(units=['km','msun','s'])
    sys.add(S)
    sys.add(P)

    Either:
    
    sys=System(stars=S,planets=P)
    
""";

In [16]:
class System(PrynglesCommon):
    
    def __init__(self,
                 units=['au','msun','yr'],
                 stars=None,planets=None,
                 rings=None,observers=None,
                 rebound=False
                ):
        
        #Behavior
        self.rebound=rebound
        self.rebound=False # Rebound not implemented yet
        self._update_rebound(units)

        #Initialize rebound
        self.update_units(units)
        
        #Initialize list of components
        self.hashes=dict()
        for kind in BODY_KINDS:
            lkind=kind.lower()
            exec(f"self.{lkind}s=[]")
            exec(f"self._update_objects('{lkind}s','{kind}',{lkind}s)")

        #Update system
        self._update_system()
    
    def _update_objects(self,attr,kind,comps):
        """Update the list of objects
        """

        if comps is not None:
            #Check if comps is a a list
            try:
                comps[0]
            except:
                comps=[comps]
            
            for comp in comps:
                #Check if primary bodies are already in lists
                if (comp.primary is not None) and (comp.primary.hash not in self.hashes):
                    raise AssertionError(f"Primary of {kind} body is not yet in the system.")
                else:
                    exec(f"self.{attr}+=[comp]")
                    if comp.kind!=kind:
                        raise AssertionError(f"You are attempting to add {kind} with a {comp.kind}")
                    self.hashes[comp.hash]=comp

    def _update_system(self):
        """Update the global properties of the system.
        """
        for kind in BODY_KINDS:
            exec(f"self.{kind}s=0")
        
        pdict=dict()
        for obj in BODY_KINDS:
            lobj=obj.lower()
            exec(f"{obj}=len(self.{lobj}s)",locals(),pdict)
            self.__dict__[f"n{lobj}s"]=pdict[obj]
        self.nbodies=self.nstars+self.nplanets+self.nrings
        
    def update_units(self,units):
        """Update units of the system
        """
        self.units=units
        self._ul,self._um,self._ut=self.units
        self._sim.units=self.units
        self.ul=rb.units.convert_length(1,self._ul,"m")
        self.um=rb.units.convert_mass(1,self._um,"kg")
        self.ut=np.sqrt(self._sim.G*self.ul**3/(self.um*GSI))

    def _update_rebound(self,units):
        """Crteate and update rebound simulation object
        """
        #Units initialization
        self._sim=rb.Simulation()
        self._sim.units=units

System.__doc__=System_doc

In [17]:
if IN_JUPYTER:
    def test_system_init(self):
        
        sys=System()
        print(sys.nbodies)
        print(sys._sim.G)
        print(sys.ul,sys.um,sys.ut)
        
        sys=System(units=['m','kg','s'])
        print(sys.nbodies)
        print(sys._sim.G)
        print(sys.ul,sys.um,sys.ut)
        
        S=Star()
        sys=System(stars=S)
        print(sys.stars)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        S2=Star()
        P=Planet(primary=S2)
        #Check that when planet does not use a star of the system an exception is raised
        self.assertRaises(AssertionError,lambda:System(stars=S,planets=P))
        
        P=Planet(primary=S)
        sys=System(stars=S,planets=P)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        R=Planet(primary=P)
        #Check that planet cannot be initialized as a planet
        self.assertRaises(AssertionError,lambda:System(stars=S,planets=P,rings=R))

        R=Ring(primary=P)
        sys=System(stars=S,planets=P,rings=R)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        
        P1=Planet(primary=S)
        P2=Planet(primary=S)
        P3=Planet(primary=S)
        sys=System(stars=S,planets=[P1,P2,P3])
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        """
        self.assertEqual(np.isclose([P.physics.wrot],
                                    [2*np.pi/PlanetDefaults.physics["prot"]],
                                    rtol=1e-7),
                         [True]*1)
        #Check exception: primary could not be different from None or Body
        self.assertRaises(AssertionError,lambda:Observer(primary="Nada"))
        """
        
    class Test(unittest.TestCase):pass    
    Test.test_system_init=test_system_init
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

0
39.476926421373
149597870700.0 1.9884754159665356e+30 31557600.0
0
6.67408e-11
1.0 1.0 1.0
{'8767372206960': <pryngles.star.Star object at 0x7f9501b23700>}
1 1 0 0 0
{'8767372206960': <pryngles.star.Star object at 0x7f9501b23700>, '8767372434161': <pryngles.planet.Planet object at 0x7f9501e9af10>}
2 1 1 0 0
{'8767372206960': <pryngles.star.Star object at 0x7f9501b23700>, '8767372434161': <pryngles.planet.Planet object at 0x7f9501e9af10>, '8767372211077': <pryngles.ring.Ring object at 0x7f9501b33850>}
3 1 1 1 0
{'8767372206960': <pryngles.star.Star object at 0x7f9501b23700>, '8767372253097': <pryngles.planet.Planet object at 0x7f9501bd7a90>, '8767372264376': <pryngles.planet.Planet object at 0x7f9501c03b80>, '8767372264358': <pryngles.planet.Planet object at 0x7f9501c03a60>}
4 1 3 0 0



----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


In [21]:
def add(self,kind=None,primary=None,orbit=None,physics=None,optics=None):
    """Add an object to the system
    
    Examples:
    
        sys=System()
        S=sys.add(kind="Star",orbit=dict(m=2))
    
    Parameters:
    
        kind: string
            Kind of object: Star, Planet, Ring, Observer.
    
        primary: Body
            Primary object.
    
        orbit: dictionary
            Set of orbital properties (see corresponding body documentation).
            
        physics: dictionary
            Set of physical properties (see corresponding body documentation).
        
        optics: dictionary
            Set of optical properties (see corresponding body documentation).
            
    Returns:
        
        Body
            Body added to the system.
    """
    if kind is None:
        raise AssertionError("You must provide a valid object kind (Star, Planet, Ring, Observer).")

    if kind not in BODY_KINDS:
        raise ValueError(f"Object kind '{kind}' is not recognized.")

    #p.e. kind = 'Star', lkind = 'star'
    kind=kind.capitalize()
    lkind=kind.lower()

    """
    This code generalize the procedure:
        if orbit is None:
            orbit = StarDefaults.orbit.copy()
        if physics is None:
            physics = StarDefaults.physics.copy()
        if optics is None:
            optics = StarDefaults.optics.copy()
    """
    pdict=dict()
    plist=[]
    for prop in PROP_TYPES:
        exec(f"{prop}={prop}",locals(),pdict)
        if pdict[prop] is None:
            exec(f"{prop}={kind}Defaults.{prop}.copy()",globals(),pdict)
        plist+=[pdict[prop]]

    """
    This code generalize the procedure
        S=Star(primary=primary,orbit=orbit,physics=physics,optics=optics)
        self._list2Objects("stars","Star",S)            
    """
    obj=eval(f"{kind}(primary=primary,orbit=plist[0],physics=plist[1],optics=plist[2])")
    self._update_objects(lkind+"s",kind,obj)
    self._update_system()
    return obj
    
System.add=add

In [22]:
if IN_JUPYTER:
    def test_system_add(self):
        sys=System()
        S=sys.add(kind="Star",orbit=dict(m=2))
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        print(sys.stars[0].orbit)
        print(S.orbit)
        
        S.update_body(orbit=dict(m=3))
        print(sys.stars[0].orbit)
                
    class Test(unittest.TestCase):pass    
    Test.test_system_add=test_system_add
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

{'8767371987261': <pryngles.star.Star object at 0x7f95017c93d0>}
1 1 0 0 0
{'m': 2}
{'m': 2}
{'m': 3}



----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [23]:
def remove(self,body_hash):
    """Remove a body from a system.

    Parameters:
        body_hash: string
            Hash of the body to remove
    
    Notes: 
        Remove eliminate body and all the childs and the childs of the childs.

    Example:
        sys=System()
        S=sys.add(kind="Star",orbit=dict(m=2))
        sys.remove(body_hash=S.hash)
        
    """
    if body_hash in self.hashes:
        obj=self.hashes[body_hash]
        lkind=obj.kind.lower()

        #Remove child objects
        for child in obj.childs:
            if child.hash in self.hashes:
                self.remove(child.hash)

        #Remove object
        exec(f"self.{lkind}s.remove(obj)")

        #Remove hash from list
        self.hashes.pop(body_hash)

        #Update system
        self._update_system()
    else:
        raise ValueError("No object with hash 'body_hash' in the system")
System.remove=remove

In [24]:
if IN_JUPYTER:
    def test_system_remove(self):
        sys=System()
        S=sys.add(kind="Star",orbit=dict(m=2))
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=S.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        sys=System()
        S=sys.add(kind="Star")
        P=sys.add(kind="Planet",primary=S)
        R=sys.add(kind="Ring",primary=P)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=S.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys=System()
        S=sys.add(kind="Star")
        P=sys.add(kind="Planet",primary=S)
        R=sys.add(kind="Ring",primary=P)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=P.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        sys=System()
        S=sys.add(kind="Star")
        P=sys.add(kind="Planet",primary=S)
        R=sys.add(kind="Ring",primary=P)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=R.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        """
        self.assertEqual(np.isclose([P.physics.wrot],
                                    [2*np.pi/PlanetDefaults.physics["prot"]],
                                    rtol=1e-7),
                         [True]*1)
        #Check exception: primary could not be different from None or Body
        self.assertRaises(AssertionError,lambda:Observer(primary="Nada"))
        """
        
    class Test(unittest.TestCase):pass    
    Test.test_system_remove=test_system_remove
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

{'8767353992208': <pryngles.star.Star object at 0x7f94f0534100>}
1 1 0 0 0
{}
0 0 0 0 0
{'8767372253133': <pryngles.star.Star object at 0x7f9501bd7cd0>, '8767372435953': <pryngles.planet.Planet object at 0x7f9501ea1f10>, '8767372253040': <pryngles.ring.Ring object at 0x7f9501bd7700>}
3 1 1 1 0
{}
0 0 0 0 0
{'8767372435884': <pryngles.star.Star object at 0x7f9501ea1ac0>, '8767372253097': <pryngles.planet.Planet object at 0x7f9501bd7a90>, '8767372443612': <pryngles.ring.Ring object at 0x7f9501ebfdc0>}
3 1 1 1 0
{'8767372435884': <pryngles.star.Star object at 0x7f9501ea1ac0>}
1 1 0 0 0
{'8767372443633': <pryngles.star.Star object at 0x7f9501ebff10>, '8767372435863': <pryngles.planet.Planet object at 0x7f9501ea1970>, '8767372267045': <pryngles.ring.Ring object at 0x7f9501c0e250>}
3 1 1 1 0
{'8767372443633': <pryngles.star.Star object at 0x7f9501ebff10>, '8767372435863': <pryngles.planet.Planet object at 0x7f9501ea1970>}
2 1 1 0 0



----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


In [3]:
def spangle_system():
    pass

In [5]:
def integrate_system(t=0):
    pass

--End--

In [ ]:
#Create a simple system
sys=System(units=["au","msun","yr"])
sys.sim.integrator='wahfast'
sys.sim.dt=0.01
#Once you create a system, a null spangler is created 
"""
You may set the observer from the very beginning
"""
sys.set_observer(n_obs=[1,1,0],alpha_obs=0)

#Add star (by default, m = 1)
S=sys.add()

#Add planet, when an object is added, it is automatically spangled
P=sys.add("Planet",radius=0.1,m=1e-3,a=1,e=0.2)

#Add moon: orbital elements are respect to equatorial plane of the primary
M=sys.add("Planet",primary=P,radius=0.01,m=1e-7,a=0.1,e=0.01)

#Add ring system
R=sys.add("Ring",primary=P,fi=1.5,fe=2.5,albedo_gray_normal=0.5,tau_gray_optical=3)

#If you change the number of spangles of an object the spanglers are reset
sys.update_body(R,nspangles=800)

#Each time an object is updated, the spangler should be rejoined and the simulation reset.

#Spangle 
#sys.spangle_system()

#You may check separately the properties of each object
R.spangler.plot3d()
R.spangler.plot_obs()

#Plot
sys.spangler.plot3d()
sys.spangler.plot_obs()

In [ ]:
print(R)

In [ ]:
#Update properties of objects, even after the system is spangled
sys.update_body(S,limb_coeffs=[0.5,0.2])
sys.update_body(R,i=30*Consts.deg,roll=60*Consts.deg)
sys.update_body(P,Prot=24*Const.hour/sys.ul)

In [ ]:
sys.set_observer(n_obs=[1,1,1],alpha_obs=0)

In [ ]:
#When you integrate you update the position of the bodies, recalculate light sources, observer configuration and 
sys.sim_integrate(0.5)

sys.spangler.plot3d()
sys.spangler.plot_obs()

In [ ]:
sys.sim_reset()

In [ ]:
#Update positions: in this way you can move the planet, for instance, around the star
sys.update_body(R,M=30*Consts.deg)

In [ ]:
"""
You may integrate without calculating photometry

Once you're ready calculate photometry
"""
sys.calc_photometry()

In [ ]:
"""
This routine only update the optical states of the spangles
"""
sys._update_states()

"""
This routine update temperatures.  This take into account the illumination history of the spangles
"""
sys._update_temperatures()